In [1]:
# Get data out of the file
def get_edges(line):
    edges = []
    line = line.translate(None,"[]")
    for token in line.split(")"):
        if token:
            values = token.split("(")[1].split(",")
            edges.append((int(values[0]),int(values[1])))
    return edges 
# make pairs (g,i)
data = []
from collections import defaultdict,deque
f = open('input.txt','r')
for line in f:
    if line != '\n':
        strings = line.split("]")
        data.append([get_edges(strings[0]),get_edges(strings[1])])
f.close()
class Graph:
    def __init__(self,edges,requests):
        self.edges = edges
        self.inputs = requests 
        self.adj_list = self.adjacency_list()
    def adjacency_list(self):
        adj_list = defaultdict(set)
        for start,end in self.edges:
            adj_list[start].add(end)
            adj_list[end].add(start)
        return adj_list
    def all_unvisited(self):
        visited = defaultdict(int)
        for v in self.adj_list:
            visited[v]=0 # we know that no vertices start from 0
        return visited
    def remove_edge(self,edge):
        s,b = edge[0],edge[1]
        # remove this guy from edges 
        tup = (s,b) if s<b else (b,s)
        if tup in self.edges:
            self.edges.remove(tup)
            self.adj_list[s].discard(b)
            self.adj_list[b].discard(s)
        # adjust the adj list
        

In [2]:
def shortest_path(graph,start,end):
        def edges_from_path(path):
            if len(path)<=1:
                return path
            edges = []
            for i in range(len(path)-1):
                tup = (path[i],path[i+1]) if path[i] < path[i+1] else (path[i+1],path[i])
                edges.append(tup)
            return edges 
        
        pending =  deque([start])
        visited = graph.all_unvisited()
        path = []
        while pending:
            current = pending.popleft()
            if current == end:
                # track back all the neighbors 
                while current != start:
                    path.append(current)
                    current = visited[current]
                path.append(current)
                path.reverse()
                return edges_from_path(path)
            
            for neighbor in graph.adj_list[current]:
                if visited[neighbor] == 0:
                    visited[neighbor] = current
                    pending.append(neighbor)
        return edges_from_path(path)

In [3]:
def graph_coloring(adj_list):
    # assume that adj_list is int:set, the vertices are marked from 1 to len(adj_list)
    total_colors = 0
    all_vertices = set(adj_list.keys())
    colors = defaultdict(int)
    colors_adj_list = defaultdict(set)
    # since this is defaultdict, it would return 0 and empty set if the emements are not present in the dict for the keys!
    # Algorithm in a nutshell:
    # http://www.geeksforgeeks.org/graph-coloring-set-2-greedy-algorithm/
    if not adj_list:
        total_colors = 1
        #pass
    else: 
        queue = deque([min(adj_list.keys())])
        #queue = deque(max([(k,len(v)) for k,v in adj_list.iteritems()],key = lambda (x,y):y)[0])
        while queue:
            c = queue.popleft()
            if not colors[c]:
                # this guy is not colored
                pick = sorted(list(all_vertices - colors_adj_list[c]))[0]
                colors[c] = pick
                total_colors = pick if total_colors < pick else total_colors
                for neighbor in adj_list[c]:
                    if not colors[neighbor]:
                        # if this guy is not colored yet
                        colors_adj_list[neighbor].add(pick)
                        queue.append(neighbor)
            if not queue:
                for vertex in adj_list.keys():
                    if colors[vertex] == 0:
                        queue.append(vertex)
                        break
        for vertex in colors.keys():
            colors[vertex]=1/float(colors[vertex])
    colors["total_colors"] = total_colors
    return colors 


In [4]:
def remove_paths(g,paths):
    for r in paths.keys():
        for edge in paths[r]:
            g.remove_edge(edge)
            
def remove_list_of_edges(g,edges):
    for edge in edges:
        g.remove_edge(edge)
            
def make_shortest_paths(g):
            shortestPaths=defaultdict(set)
            for s,t in g.inputs:
                shortestPaths[(s,t)] = set(shortest_path(g,s,t))
            return shortestPaths
        
def make_graph_of_demands(demand_paths):
        translate = sorted(demand_paths.keys()) # use indices as back and forth translation 
        adj_list = defaultdict(set)
        for i in range(len(translate)):
            for j in range(i+1,len(translate)):
                if i != j and demand_paths[translate[i]].intersection(demand_paths[translate[j]]):
                    adj_list[i+1].add(j+1)
                    adj_list[j+1].add(i+1)
        return adj_list

def number_of_different_wavelengths(adj_list):
    #adj_list = make_graph_of_demands(g)
    #edges = edges_from_adj_list(adj_list)
    color_map =graph_coloring(adj_list)
    #draw_graph(edges,color_map)
    return color_map['total_colors']

########################################################################
## USAGE for unit testing only###
def edges_from_adj_list(adj_list):
    edges = []
    for vertex in adj_list:
        for neighbor in adj_list[vertex]:
            if vertex < neighbor:
                edges.append((vertex,neighbor))
    return edges
import networkx as nx
import matplotlib.pyplot as plt
def draw_graph(edges_list, values_map = None):
    G=nx.Graph()
    G.add_edges_from(edges_list)
    if not values_map:
        nx.draw(G,with_labels = True,node_color='Aqua')
    else:
        values = [values_map[node] for node in G.nodes()]
        nx.draw(G, cmap=plt.get_cmap('cool'), node_color=values,with_labels = True)
    plt.show()

In [5]:
def modify_old_paths(oldpaths,new_graph):
    requests = sorted(oldpaths.keys())
    for i,r in enumerate(requests):
        flag = 0 
        for j in range(i):
            if oldpaths[requests[j]].intersection(oldpaths[r]) and flag == 0:
                #find a new path for r in the new graph if possible 
                possible_path = set(shortest_path(new_graph,r[0],r[1]))
                if possible_path:
                    # yes there is a path for the request r in the new_graph, set this as a pat andremove edges from new graph
                    #oldpaths[r]= possible_path
                    oldpaths.pop(r,None)
                    flag = 1
                    remove_list_of_edges(new_graph,possible_path)
                    #print r,possible_path

In [6]:
def remove_one_at_a_time(oldpaths,graph):
    requests = sorted(oldpaths.keys())
    for r in requests:
        possible_path = set(shortest_path(graph,r[0],r[1]))
        if possible_path:
            oldpaths[r] = possible_path
            remove_list_of_edges(graph,possible_path)

In [7]:
# read from another file 
ratios = [1.0654205607476634, 0.3644859813084112, 0.18691588785046728, 0.102803738317757, 1.4042553191489362, 0.4148936170212766, 0.2127659574468085, 0.11702127659574468, 0.4253731343283582, 0.373134328358209, 0.14925373134328357, 0.08208955223880597, 2.980769230769231, 1.0384615384615385, 0.38461538461538464, 0.21153846153846154, 4.702702702702703, 1.4054054054054055, 0.5405405405405406, 0.2972972972972973, 1.9610389610389611, 0.5064935064935064, 0.2597402597402597, 0.14285714285714285, 2.140845070422535, 0.5492957746478874, 0.28169014084507044, 0.15492957746478872, 1.305263157894737, 0.4105263157894737, 0.21052631578947367, 0.11578947368421053, 1.527027027027027, 0.8648648648648649, 0.2702702702702703, 0.14864864864864866, 0.4258064516129032, 0.25161290322580643, 0.12903225806451613, 0.07096774193548387, 0.9137931034482759, 0.33620689655172414, 0.1724137931034483, 0.09482758620689655, 3.7291666666666665, 0.8125, 0.4166666666666667, 0.22916666666666666, 2.09375, 4.28125, 0.625, 0.34375, 0.3416149068322981, 0.2422360248447205, 0.12422360248447205, 0.06832298136645963, 0.3225806451612903, 0.25161290322580643, 0.12903225806451613, 0.07096774193548387, 0.8932038834951457, 0.5145631067961165, 0.1941747572815534, 0.10679611650485436, 0.19801980198019803, 1.108910891089109, 0.19801980198019803, 0.10891089108910891, 0.6126126126126126, 0.5765765765765766, 0.18018018018018017, 0.0990990990990991, 5.2631578947368425, 5.7368421052631575, 1.0526315789473684, 0.5789473684210527, 1.9473684210526316, 8.473684210526315, 1.0526315789473684, 0.5789473684210527, 0.36538461538461536, 0.25, 0.1282051282051282, 0.07051282051282051, 0.4791666666666667, 0.2708333333333333, 0.1388888888888889, 0.0763888888888889, 1.0357142857142858, 0.3482142857142857, 0.17857142857142858, 0.09821428571428571, 0.6016949152542372, 0.4661016949152542, 0.1694915254237288, 0.09322033898305085, 2.0, 0.5131578947368421, 0.2631578947368421, 0.14473684210526316, 0.09580838323353294, 0.2934131736526946, 0.11976047904191617, 0.0658682634730539, 1.7272727272727273, 1.7636363636363637, 0.36363636363636365, 0.2, 0.4027777777777778, 0.2777777777777778, 0.1388888888888889, 0.0763888888888889, 0.10294117647058823, 0.5955882352941176, 0.14705882352941177, 0.08088235294117647, 3.6530612244897958, 0.7959183673469388, 0.40816326530612246, 0.22448979591836735, 4.566666666666666, 2.7, 0.6666666666666666, 0.36666666666666664, 0.6363636363636364, 1.4675324675324675, 0.2597402597402597, 0.14285714285714285, 5.7368421052631575, 5.315789473684211, 1.0526315789473684, 0.5789473684210527, 6.2105263157894735, 4.842105263157895, 1.0526315789473684, 0.5789473684210527, 0.5694444444444444, 0.2708333333333333, 0.1388888888888889, 0.0763888888888889, 0.5277777777777778, 0.2708333333333333, 0.1388888888888889, 0.0763888888888889, 0.9473684210526315, 8.789473684210526, 1.5263157894736843, 0.5789473684210527, 0.5172413793103449, 0.2689655172413793, 0.13793103448275862, 0.07586206896551724, 0.37735849056603776, 0.24528301886792453, 0.12578616352201258, 0.06918238993710692, 0.30392156862745096, 1.0098039215686274, 0.19607843137254902, 0.10784313725490197, 1.9230769230769231, 5.769230769230769, 0.7692307692307693, 0.4230769230769231, 8.052631578947368, 3.0, 1.0526315789473684, 0.5789473684210527, 2.4634146341463414, 2.6341463414634148, 0.4878048780487805, 0.2682926829268293, 0.0625, 2.515625, 0.3125, 0.171875, 1.0, 8.894736842105264, 1.368421052631579, 0.5789473684210527, 0.22699386503067484, 0.2392638036809816, 0.12269938650306748, 0.06748466257668712, 1.736842105263158, 8.631578947368421, 1.0526315789473684, 0.5789473684210527, 0.4125874125874126, 0.2727272727272727, 0.13986013986013987, 0.07692307692307693, 6.88, 1.76, 0.8, 0.44, 0.3761467889908257, 0.8073394495412844, 0.1834862385321101, 0.10091743119266056, 1.3846153846153846, 3.769230769230769, 0.5128205128205128, 0.28205128205128205, 0.15060240963855423, 0.26506024096385544, 0.12048192771084337, 0.06626506024096386, 1.9154929577464788, 0.704225352112676, 0.28169014084507044, 0.15492957746478872, 0.24369747899159663, 0.7310924369747899, 0.16806722689075632, 0.09243697478991597, 0.5797101449275363, 0.2826086956521739, 0.14492753623188406, 0.07971014492753623, 1.9610389610389611, 0.5064935064935064, 0.2597402597402597, 0.14285714285714285, 2.272727272727273, 0.6666666666666666, 0.30303030303030304, 0.16666666666666666, 7.2631578947368425, 3.789473684210526, 1.0526315789473684, 0.5789473684210527, 0.6347826086956522, 0.4956521739130435, 0.17391304347826086, 0.09565217391304348, 5.2105263157894735, 5.842105263157895, 1.0526315789473684, 0.5789473684210527, 1.22, 0.39, 0.2, 0.11, 4.944444444444445, 1.3611111111111112, 0.5555555555555556, 0.3055555555555556, 4.157894736842105, 6.684210526315789, 1.0526315789473684, 0.5789473684210527, 0.8831168831168831, 1.2597402597402598, 0.2597402597402597, 0.14285714285714285, 6.7894736842105265, 4.2631578947368425, 1.0526315789473684, 0.5789473684210527, 0.34355828220858897, 0.2392638036809816, 0.12269938650306748, 0.06748466257668712, 0.8923076923076924, 1.8153846153846154, 0.3076923076923077, 0.16923076923076924, 0.40939597315436244, 0.26174496644295303, 0.1342281879194631, 0.0738255033557047, 1.855263157894737, 0.5394736842105263, 0.2631578947368421, 0.14473684210526316, 3.326923076923077, 0.7692307692307693, 0.38461538461538464, 0.21153846153846154, 0.8518518518518519, 0.4537037037037037, 0.18518518518518517, 0.10185185185185185, 2.775, 2.525, 0.5, 0.275, 0.7931034482758621, 2.310344827586207, 0.3448275862068966, 0.1896551724137931, 5.526315789473684, 5.526315789473684, 1.0526315789473684, 0.5789473684210527, 1.5, 2.36, 0.4, 0.22, 5.631578947368421, 5.368421052631579, 1.0526315789473684, 0.5789473684210527, 7.894736842105263, 3.1578947368421053, 1.0526315789473684, 0.5789473684210527, 0.5147058823529411, 0.2867647058823529, 0.14705882352941177, 0.08088235294117647, 2.2941176470588234, 0.5735294117647058, 0.29411764705882354, 0.16176470588235295, 4.7894736842105265, 6.2105263157894735, 1.0526315789473684, 0.5789473684210527, 1.1818181818181819, 0.6590909090909091, 0.22727272727272727, 0.125, 0.4426229508196721, 2.3934426229508197, 0.32786885245901637, 0.18032786885245902, 0.5789473684210527, 8.894736842105264, 1.736842105263158, 0.5789473684210527, 7.7368421052631575, 3.3157894736842106, 1.0526315789473684, 0.5789473684210527, 1.4264705882352942, 1.2352941176470589, 0.29411764705882354, 0.16176470588235295, 8.421052631578947, 2.6315789473684212, 1.0526315789473684, 0.5789473684210527, 0.6060606060606061, 0.29545454545454547, 0.15151515151515152, 0.08333333333333333, 1.268041237113402, 0.4020618556701031, 0.20618556701030927, 0.1134020618556701, 0.41139240506329117, 0.2468354430379747, 0.12658227848101267, 0.06962025316455696, 0.2119205298013245, 0.3509933774834437, 0.13245033112582782, 0.0728476821192053, 0.7317073170731707, 0.3170731707317073, 0.16260162601626016, 0.08943089430894309, 0.8181818181818182, 0.43636363636363634, 0.18181818181818182, 0.1, 0.0975609756097561, 1.7073170731707317, 0.24390243902439024, 0.13414634146341464, 5.083333333333333, 1.2222222222222223, 0.5555555555555556, 0.3055555555555556, 0.5473684210526316, 0.9789473684210527, 0.21052631578947367, 0.11578947368421053, 0.16666666666666666, 6.0, 0.8666666666666667, 0.36666666666666664, 2.0526315789473686, 3.3947368421052633, 0.5263157894736842, 0.2894736842105263, 1.0133333333333334, 1.24, 0.26666666666666666, 0.14666666666666667, 0.5034013605442177, 0.2653061224489796, 0.1360544217687075, 0.07482993197278912, 2.015151515151515, 0.8484848484848485, 0.30303030303030304, 0.16666666666666666]

In [ ]:
#driver1
f1 = open('G1_wavelengths_ratios','w')
f2 = open('ls1_wavelengths__ratios','w')
for i, line in enumerate(data):
    #print line[0],"\n",line[1]
    #line[0] is the graph, line[1] is the requests 
    g = Graph(line[0],line[1])
    #common = ";"+str(len(g.adj_list))+";"+str(len(g.edges))+";"+str(len(g.inputs))+"\n"
    initial_paths  = make_shortest_paths(g)
    w1 = number_of_different_wavelengths(make_graph_of_demands(initial_paths))
    
    remove_paths(g,initial_paths)
    modify_old_paths(initial_paths,g)
    w2 = number_of_different_wavelengths(make_graph_of_demands(initial_paths))

    result1 = str(w1)+","+str(ratios[i])+"\n"
    result2 = str(w2)+","+str(ratios[i])+"\n"
    f1.write(result1)
    f2.write(result2)
f1.close()
f2.close()
    #print result

In [9]:
#driver2
f3 = open('G3_wavelengths_ratios','w')
for i,line in enumerate(data):
    g = Graph(line[0],line[1])
    paths  = make_shortest_paths(g)
    remove_one_at_a_time(paths,g)
    w3 = number_of_different_wavelengths(make_graph_of_demands(paths))
    result3 = str(w3)+","+str(ratios[i])+"\n"
    f3.write(result3)
f3.close()

line = [[(14, 17), (7, 12), (1, 17), (17, 20), (1, 6), (1, 11), (6, 7), (12, 17), (13, 20), (9, 14), (4, 5), (10, 13), (16, 19), (2, 17), (17, 18), (3, 18), (10, 14), (9, 19), (7, 8), (2, 18), (8, 9), (6, 14), (3, 6), (1, 10), (4, 11), (3, 5), (9, 13), (4, 6), (5, 7), (5, 20), (16, 20), (3, 15), (4, 8), (5, 13), (8, 19), (9, 18), (7, 11), (11, 19), (16, 17), (2, 19), (1, 14), (8, 10), (4, 13), (6, 15), (12, 14), (13, 15), (10, 16), (13, 18), (3, 4), (9, 12), (5, 9), (7, 16), (8, 14), (2, 9), (5, 12), (10, 12), (9, 17), (11, 18), (16, 18), (17, 19), (5, 15), (3, 19), (8, 17), (6, 12), (6, 19), (12, 15), (11, 17), (3, 8)], [(1, 2), (1, 4), (1, 5), (1, 9), (1, 12), (1, 16), (2, 6), (2, 8), (2, 12), (3, 9), (3, 12), (3, 17), (4, 9), (4, 12), (4, 16), (5, 8), (5, 10), (5, 14), (5, 18), (6, 8), (6, 9), (6, 16), (6, 17), (7, 15), (7, 19), (8, 12), (8, 16), (9, 15), (10, 18), (11, 12), (12, 13), (12, 18), (12, 20), (13, 17), (14, 16), (14, 18), (15, 17), (15, 18), (15, 20)]]

g = Graph(line[0],line[1])

In [ ]:
initial_paths  = make_shortest_paths(g)
initial_wavelengths = number_of_different_wavelengths(make_graph_of_demands(initial_paths))
w1 = initial_wavelengths
remove_paths(g,initial_paths)
modify_old_paths(initial_paths,g)
wavelengths = number_of_different_wavelengths(make_graph_of_demands(initial_paths))
w2 = wavelengths
print w1,w2